In [44]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import csv
import re

In [45]:
# Khởi tạo driver một lần
driver = webdriver.Chrome()
data = []

# Lặp qua tất cả 30 trang
for page in range(1, 31):
    print(f"Đang crawl trang {page}/30...")
    
    # Truy cập từng trang
    url = f"https://www.galaxycine.vn/binh-luan-phim/?page={page}&type=review&sortBy=view"
    driver.get(url)
    time.sleep(3)  # chờ JS load
    
    # Parse HTML cho trang hiện tại
    soup = BeautifulSoup(driver.page_source, "html.parser")
    reviews = soup.find_all("ul", class_="post__list")
    
    # Trích xuất dữ liệu từ trang hiện tại
    for review_list in reviews:
        items = review_list.find_all("li", class_="mb-2")
        
        for item in items:
            # Trích xuất Title
            title_link = item.find("a", class_="text-black-10 text-sm md:text-[#333] md:text-lg " 
                                               "font-semibold md:font-semibold not-italic transition-all " 
                                               "duration-500 ease-in-out item__title")
            if title_link:
                full_title = title_link.get("title") or title_link.text.strip()
                
                # Lấy phần sau [Review] và trước :
                if "[Review]" in full_title or "[Preview]" in full_title:
                    after_review = full_title.split("[Review]")[1].strip() if "[Review]" in full_title else full_title.split("[Preview]")[1].strip()
                    if ":" in after_review:
                        title = after_review.split(":")[0].strip()
                else:
                    title = full_title
            else:
                title = "N/A"
            
            # Trích xuất Views
            views_element = item.find("button", class_="text-xs bg-grey-120 rounded mr-2 py-1 px-5")
            views = views_element.text.strip() if views_element else "0"
            
            # Trích xuất Rating
            rating_element = item.find("span", class_="inline-block mr-1") or item.find("div", class_="rating")
            rating = rating_element.text.strip() if rating_element else "N/A"
            
            # Trích xuất Votes - chỉ lấy phần số
            votes_element = item.find("span", class_="inline-block text-sm text-grey-40")
            if votes_element:
                votes_text = votes_element.text.strip()
                # Sử dụng regex để tìm số đầu tiên
                votes_match = re.search(r'\d+', votes_text)
                votes = votes_match.group() if votes_match else "0"
            else:
                votes = "0"

            print(f"Title: {title}")
            print(f"Views: {views}")
            print(f"Votes: {votes}")
            print(f"Rating: {rating}")
            print("------------------------------------\n")
            
            if (votes != "0" and 
                rating != "N/A" and 
                rating.strip() != "" and 
                title != "N/A" and 
                views != "0"):
                
                data.append({
                    "Title": title,
                    "Views": views,
                    "Votes": votes,
                    "Rating": rating,
                })
    
    # Nghỉ một chút giữa các request để tránh bị block
    time.sleep(2)

# Đóng driver sau khi crawl xong
driver.quit()

print(f"Đã crawl xong! Tổng cộng {len(data)} bài review từ 30 trang.")

Đang crawl trang 1/30...
Title: Em Chưa 18
Views: 145669
Votes: 88
Rating: 7.92
------------------------------------

Title: Lật Mặt 2
Views: 114812
Votes: 32
Rating: 8.74
------------------------------------

Title: Guardians Of The Galaxy Vol.2
Views: 84823
Votes: 26
Rating: 7.38
------------------------------------

Title: Your Name
Views: 68094
Votes: 20
Rating: 9.14
------------------------------------

Title: Tây Du Ký 2
Views: 67146
Votes: 16
Rating: 7.53
------------------------------------

Title: Tấm Cám – Chuyện Chưa Kể
Views: 66983
Votes: 43
Rating: 7.95
------------------------------------

Title: Phù Thủy Tối Thượng
Views: 66969
Votes: 25
Rating: 9.23
------------------------------------

Title: Me Before You
Views: 64522
Votes: 24
Rating: 8.75
------------------------------------

Title: Biệt Đội Cảm Tử
Views: 63118
Votes: 47
Rating: 8.53
------------------------------------

Title: Nắng
Views: 61919
Votes: 22
Rating: 9.33
------------------------------------

Title: Mỹ 

In [46]:
with open('film_review_only_voted.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Title', 'Views', 'Votes', 'Rating'])
    for item in data:
        row = [
            item['Title'],
            item['Views'],
            item['Votes'],
            item['Rating'],
        ]
        writer.writerow(row)